In [ ]:
# import os
# from PIL import Image
# from transformers import ViTImageProcessor, AutoImageProcessor
# import cv2

# data = {"img": [], "label": []}

# # change label to index
# label_dictionary = {"小白": 0,
#                     "巧巧": 1,
#                     "冏媽": 2,
#                     "乖狗": 3,
#                     "花捲": 4,
#                     "超人": 5,
#                     "黑面": 6,
#                     "橘子": 7,
#                     "黑胖": 8,
#                     "不是校狗": 9
#                     }  # 用於存放標籤與索引的字典


# # 資料夾的根路徑
# root_folder = "狗狗照片"  # 更換為你的實際資料夾路徑

# # 遍歷根資料夾下的所有子資料夾
# for root, dirs, files in os.walk(root_folder):
#     for dog_name in dirs:
#         dog_folder = os.path.join(root, dog_name)
#         # 遍歷狗的資料夾下的所有圖片文件
#         for filename in os.listdir(dog_folder):
#             if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
#                 img_path = os.path.join(dog_folder, filename)
#                 label = label_dictionary[dog_name]
#                 # 讀取圖片並轉換為 RGB 格式
#                 img = cv2.imread(img_path)
#                 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#                 # Resize the image to 224x224 pixels
#                 img = Image.fromarray(img)
#                 img = img.resize((224, 224))

#                 data["img"].append(img)
#                 data["label"].append(label)

# # Example of using the ViT image processor
# model_name = 'google/vit-large-patch16-224-in21k'
# image_processor = AutoImageProcessor.from_pretrained(model_name)

# # Preprocess the image using the ViT image processor
# inputs = image_processor(images=data["img"], return_tensors="pt", padding="max_length", max_length=224, truncation=True)

# # Display the processed inputs
# print(inputs)


In [ ]:
# # random split data into trainset and vaildset and shuffle data and save using pickle
# # notice the datastructure of data 

# import pickle
# from sklearn.model_selection import train_test_split

# train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(inputs["pixel_values"], data["label"], test_size=0.2, random_state=42)

# train_data = {"img": train_inputs, "label": train_labels}
# valid_data = {"img": valid_inputs, "label": valid_labels}

# with open("train_data.pickle", "wb") as f:
#     pickle.dump(train_data, f)
    
# with open("valid_data.pickle", "wb") as f:
#     pickle.dump(valid_data, f)


In [2]:
import torch
# use cuda:0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [3]:
import torch
import pickle
from transformers import ViTForImageClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset

# Load the train and vaild
with open("train_data.pickle", "rb") as f:
    train_data = pickle.load(f)

with open("valid_data.pickle", "rb") as f:
    valid_data = pickle.load(f)

# Convert the dataset into torch tensors
train_inputs = torch.tensor(train_data["img"])
train_labels = torch.tensor(train_data["label"])
valid_inputs = torch.tensor(valid_data["img"])
valid_labels = torch.tensor(valid_data["label"])

# Create the TensorDataset
train_dataset = TensorDataset(train_inputs, train_labels)
valid_dataset = TensorDataset(valid_inputs, valid_labels)

# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)



/tmp/ipykernel_4863/2907847089.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_data["img"])
/tmp/ipykernel_4863/2907847089.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  valid_inputs = torch.tensor(valid_data["img"])


In [4]:


# Define the model and move it to the GPU
num_classes = 10  # Adjust according to your classification task
model_name = 'google/vit-large-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(model_name, num_labels=num_classes)
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Fine-tune the model
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        # Move batch to the GPU
        batch = [r.to(device) for r in batch]

        # Unpack the inputs from our dataloader
        inputs, labels = batch

        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, labels=labels)

        # Compute loss
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update the loss
        total_loss += loss.item()
        
    # Get the average loss for the entire epoch
    avg_loss = total_loss / len(train_loader)
    
    # Print the loss
    print('Epoch:', epoch + 1, 'Training Loss:', avg_loss)
    
    
# Evaluate the model on the validation set
model.eval()
total_correct = 0

for batch in valid_loader:
    # Move batch to the GPU
    batch = [t.to(device) for t in batch]

    # Unpack the inputs from our dataloader
    inputs, labels = batch

    # Forward pass
    with torch.no_grad():
        outputs = model(inputs)

    # Get the predictions
    predictions = torch.argmax(outputs.logits, dim=1)

    # Update the total correct
    total_correct += torch.sum(predictions == labels)
    
# Calculate the accuracy
accuracy = total_correct / len(valid_dataset)
print('Validation accuracy:', accuracy.item())


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/pms/miniconda3/envs/textgen/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1 Training Loss: 0.4796100751984687
Epoch: 2 Training Loss: 0.049190593742957665
Epoch: 3 Training Loss: 0.1343484412553528
Epoch: 4 Training Loss: 0.027254830753903777
Epoch: 5 Training Loss: 0.004159979653426461
Validation accuracy: 0.9841270446777344


In [5]:
# Save the model
model.save_pretrained("model")


In [9]:
# test model with one image (test.jpg)
from PIL import Image
import torch
import cv2
from transformers import ViTForImageClassification, ViTFeatureExtractor, AutoImageProcessor

# change label to index
label_dictionary = {"小白": 0,
                    "巧巧": 1,
                    "冏媽": 2,
                    "乖狗": 3,
                    "花捲": 4,
                    "超人": 5,
                    "黑面": 6,
                    "橘子": 7,
                    "黑胖": 8,
                    "不是校狗": 9
                    }  # 用於存放標籤與索引的字典

# Load the model
# model = ViTForImageClassification.from_pretrained("model")
model_name = 'google/vit-large-patch16-224-in21k'
image_processor = AutoImageProcessor.from_pretrained(model_name)

# # Preprocess the image using the ViT image processor

# Load the image
img_path = "test.jpg"

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize the image to 224x224 pixels
img = Image.fromarray(img)
img = img.resize((224, 224))

# img to tensor
# Preprocess the image and generate features
inputs = image_processor(images=img, return_tensors="pt")
inputs.to(device)
outputs = model(**inputs)
logits = outputs.logits
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", predicted_class_idx)
print("Predicted class name:", list(label_dictionary.keys())[list(label_dictionary.values()).index(predicted_class_idx)])


Predicted class: 7
Predicted class name: 橘子


In [ ]:
import torch
import pickle
from transformers import ViTForImageClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset

# Load the model
model_name = 'model'
model = ViTForImageClassification.from_pretrained(model_name, num_labels=10)

# Load the test data
# Load the image
img_path = "test.jpg"

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize the image to 224x224 pixels
img = Image.fromarray(img)
img = img.resize((224, 224))

# img to tensor
# Preprocess the image and generate features
inputs = image_processor(images=img, return_tensors="pt")
inputs.to(device)
outputs = model(**inputs)
logits = outputs.logits
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])